In [11]:
import tensorflow as tf
import cv2
import numpy as np
import os
from pygame import mixer
import time

In [12]:
# Load Haar cascades and the model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = tf.keras.models.load_model('save_models/new_best_model.h5')

In [14]:
# Initialize the alarm sound
mixer.init()
sound = mixer.Sound('sounds/alarm2.wav')
cap = cv2.VideoCapture(0)

score = 0
drowsy_threshold = 15
is_playing = False  # To track sound state
cool_down_timer = 0  # Time tracking variable
cool_down_duration = 5  # Cool-down duration in seconds

# UI parameters
overlay_color = (0, 0, 0)  # Black overlay for status
text_color = (255, 255, 255) 
alert_color = (0, 0, 255) 
awake_color = (0, 255, 0) 
font = cv2.FONT_HERSHEY_SIMPLEX

def draw_header_bar(frame, text="Drowsiness Detection"):
    height, width = frame.shape[:2]
    bar_height = 50
    cv2.rectangle(frame, (0, 0), (width, bar_height), (50, 50, 50), -1)
    cv2.putText(frame, text, (10, 35), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

def draw_status_overlay(frame, status_text, color=(255,255,255)):
    height, width = frame.shape[:2]
    overlay_height = 60
    sub_img = frame[height - overlay_height:height, 0:width]
    overlay = sub_img.copy()
    cv2.rectangle(overlay, (0, 0), (width, overlay_height), overlay_color, -1)
    cv2.addWeighted(overlay, 0.6, sub_img, 0.4, 0, sub_img)
    frame[height - overlay_height:height, 0:width] = sub_img
    cv2.putText(frame, status_text, (10, height - 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 2)

def draw_alert_overlay(frame):
    height, width = frame.shape[:2]
    cv2.rectangle(frame, (0, 0), (width, 50), alert_color, -1)
    cv2.putText(frame, "ALERT! You are Drowsy!", (10, 35), font, 1, text_color, 2)

def draw_drowsiness_bar(frame, score, max_score=30):
    height, width = frame.shape[:2]
    bar_x = width - 30
    bar_y_start = 60
    bar_height = 200
    bar_thickness = 20
    
    cv2.rectangle(frame, (bar_x, bar_y_start), (bar_x+bar_thickness, bar_y_start+bar_height), (50, 50, 50), -1)
    
    filled_height = int((score / max_score) * bar_height)
    filled_color = alert_color if score > drowsy_threshold else awake_color
    cv2.rectangle(frame, (bar_x, bar_y_start + (bar_height - filled_height)), 
                  (bar_x+bar_thickness, bar_y_start + bar_height), filled_color, -1)
    
    cv2.putText(frame, "Score", (bar_x - 60, bar_y_start + 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, text_color, 2)
    cv2.putText(frame, str(score), (bar_x - 40, bar_y_start + bar_height + 20), 
                cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, text_color, 2)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces and eyes
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    # Draw UI elements
    draw_header_bar(frame)
    draw_drowsiness_bar(frame, score)

    # Initialize a variable to show eye prediction text (in case no eyes found)
    pred_text = ""

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), color=(255, 0, 0), thickness=2)

    # Process only if we have detected eyes
    # For simplicity, we'll consider the first detected eye as representative
    # To be more accurate, consider averaging multiple eyes if available.
    if len(eyes) > 0:
        (ex, ey, ew, eh) = eyes[0]
        eye_roi = frame[ey:ey+eh, ex:ex+ew]
        eye_roi = cv2.resize(eye_roi, (80, 80))
        eye_roi = eye_roi / 255.0
        eye_roi = eye_roi.reshape(1, 80, 80, 3)

        # Predict using the model
        prediction = model.predict(eye_roi, verbose=0)
        closed_prob = np.round(float(prediction[0][0]), 4)
        open_prob = np.round(float(prediction[0][1]), 4)

        pred_text = f"Closed: {closed_prob:.2f}, Open: {open_prob:.2f}"
        draw_status_overlay(frame, pred_text, text_color)

        # Handle drowsiness detection
        if closed_prob > 0.30:
            score += 1
            draw_status_overlay(frame, 'Status: DROWSY!', alert_color)
            if score > drowsy_threshold:
                draw_alert_overlay(frame)
                if not is_playing and time.time() - cool_down_timer > cool_down_duration:
                    sound.play(loops=-1)  # Play continuously
                    is_playing = True
        elif open_prob > 0.90:
            score -= 1
            score = max(score, 0)
            draw_status_overlay(frame, 'Status: AWAKE', awake_color)
            if is_playing:  # Stop sound if driver opens eyes
                sound.stop()
                is_playing = False
                cool_down_timer = time.time()  # Start the cool-down timer
        else:
            # If probabilities are inconclusive, show uncertain
            draw_status_overlay(frame, "Status: Uncertain", (255, 255, 0))
    else:
        draw_status_overlay(frame, "No eyes detected", (0, 255, 255))

    cv2.imshow('Drowsiness Detection', frame)
    # Quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [13]:
cap.release()
cv2.destroyAllWindows()